# Jackpot – a highly scalable, generalised Potts Model simulator

Submitted: 2023-05-01

## Introduction
The understanding of phase transitions in physical systems is often aided simulations of _lattice models_, with perhaps the most famous example being the revered _Ising Model_,
which is often formulated as a two-dimensional square lattice of of _spins_ that may adopt
one of two states ${-1, 1}$. Remarkably, when neighbouring spins on the lattice are taken to
interact with a coupling strength $J$ and the system is evolved in a manner that satisfies the principle of _detailed balance_, a phase transition be observed at a critical point as defined by an independent parameter of the system, which will be the inverse temperature $β ≡ \frac{1}{k_B T}$ when working with Boltzmann distributions.

The Ising Model is further generalised by the _Potts Model_, which again may be generalised into the _$N$-Vector Model_, which also encapsulates the _Heisenberg Model_ and _XY Model_, all of which are important models in statistical mechanics and condensed matter physics, though they occassionally find also find uses in network theory and quantitative sociology (van der Maas et al., 2020).

Evolving a system such that it reaches an equilibrium position and subsequently sampling different uncorrelated states of the equilibrated system proves to be a very computationally intensive process. A variety of different algorithms exist that obey the detailed balance criterion while trying to propagate and sample the model states as efficiently as possible, where a notable example of a _local algorithm_ is the famous _Metropolis-Hastings algorithm_, though it is often preferable to for larger lattice sizes to work with a _cluster algorithm_ that is able to change multiple lattice sites per iteration, with the _Wolff algorithm_ being a common choice.

_Jackpot_ is a framework for working with a variety of different lattice models and algorithms in a highly scalable manner and makes use of the XLA (_Accelerated Linear Algebra_, (XLA, Google, n.d.)) compiler from Google in order to target fast and energy-efficient accelerator hardware such as NVIDIA GPU's and Google TPU's (_Tensor Processing Units_, (TPU, Google, n.d.)) in addition to traditional CPU-based execution.

## Theory - Physics

### A Brief Introduction to Lattice Models
While this project is primarily focused on the computational approaches to working with lattice models and how to best solve the problems around scaling them, it is important to have a thorough understanding of the physical behaviour of the system we are trying to simulate. A very thorough description of the Ising Model is given in the the somewhat ironically named publication "_A Brief Account of the Ising and Ising-like Models_" by Jozef Strečka and Michal Jaščur (Strecka and Jaščur, 2015), though much of this is concerned with the more analytically tractable _mean field_ approaches to solving the Ising Model, whereas we wish to explore the _exact field_ version of the model by leveraging our computational implementation.

While the Ising Model is most commonly studied in two dimensions, we will do away with this constraint and develop our understanding using an arbitrary number of dimensions, drawing examples from a two-dimensional system where these may benefit our understanding.

Having lifted the constraint of two dimensions, we can further generalise the Ising Model by instead moving to the _Potts Model_ where we no longer limit the number of possible spin states to just two, but rather $q$ different discrete spin states. 

Additionally, we can introduce more interaction parameters such as an external interacting field that encourages spins to align in a particular spin state over another or we can have interaction effects that go beyond merely the nearest neighbour on the lattice, which itself does not have to be hypercubic, but may take any crystal lattice structure and adopt an arbitrarily complex _motif_.

While we may wish to allow for all of these generalisations and more in our framework, we
will restrict ourselves to models on hypercubic lattices with one a site in the motif. _Jackpot_ is written in a way where these generalisations can be lifted, though not as easily and interchangibly as changing the model or evolution algorithm.

#### An Aside on the Generality of Lattice Models
The description of lattice models above is heavily tinted by the physical framework within which lattice models often find their application. For example, one might want to avoid referring to the spin states as just that and instead adopt the more general terminology of lattice site states if investigating the model outside of a physical perspective, though we are physicists and will allow our familiar terminology here.

Indulge me as we venture away from the physical part of physics and instead apply our
physical intuition to another important dynamic system: society.

It is worthwhile keeping in mind that these models do have uses outside of physics and every physical, thermodynamic phenomenon must therefore, interestingly, have an equivalent quantity in whatever domain the model is applied to. For example, if the Ising Model is applied to party political affiliation along an axis mapping the wealth distributive policy of a system, one might adopt the public's political engagement as the temperature-like variable and find that systems with lower democratic activity will equilibrate into either a capitalist "phase" or a socialist "phase" below a critical point. This is analogous to how the magnetisation density is bifurcated below the critical temperature in the physical Ising Model – political systems tend to be somewhat more dynamic and harder to make fit into quantitative models than physical systems, though anecdotally the American two-party system does seem to be "too cold" to exist in a single phase – one might wonder what intensive variable might be tuned to overcome the the phenomenon of political polarisation – Perhaps the Information Age has lead to an increase in the linear interaction parameter $J$, which would in turn increase the critical "temperature" $T_c$ needed to prevent precipitation.

Be warned: when you allow yourself to observe physics outside the traditional domain of the discipline, its models start cropping up in the most peculiar places. 

As we recover from the frightful prospect of having briefly turned into _social scientists_ – two terms that normally exist in separate phases when interpreted either as a behavioural statement or a disciplinary description – we quickly return to the familiar embrace of the mathematical framework of statistical physics.

While a full regurgitation is beyond the scope of this report, we will introduce our lattice models in some detail here, starting with the Ising Model

### The Ising Model

We construct a an array of size $L^D$ that represents a trunctated hypercubic lattice with each element of the array representing a single lattice site, indexed by $i$, which may adopt values $s_i ∈ {±0.5}$ (note that this is sometimes taken to be ${±1}$ in the literature, which leads to a factor $2$ difference in the critical temperature value). Conceptually it is helpful to arrange our array as an ND-array – that is, an array of $D$ axes, in which case our indexing variable $i$ becomes a set of numbers that uniquely index each site.

We can then give a geometrical meaning to our array and index by declaring that indices that vary by $±1$ along a single axis are said to be _neighbours_ – in the most general case these would be connected vertices on an arbitrary graph. It now becomes difficult not to imagine our array as an $D$-dimensional primitive hypercubic crystal, which is indeed a helpful picture to keep in mind.

Next we want to introduce an interaction Hamiltonian that describes how the energy of our system arises as a function of the spins and their interactions with their neighbours. For the Ising Model a generalised interaction Hamiltonian could look like (Taheridehkordi and Zivieri, 2020):

$$
𝒣(s) =
-\sum_{<ij>} J_{ij} s_i s_j
-\sum_{<ij>} K_{ij} s_i^2 s_j^2
-\sum_{i=0}^{L^D - 1} D_i s_i^2
-\sum_{<ij>} L_{ij} (s_i^2 s_j + s_i s_j^2)
-\sum_{i=0}^{L^D - 1} μ_i H_i s_i
$$

Where $<ij>$ sums over neighbouring sites only and $i,j$ are taken to the the "flat" indices of our array – that is, for the purposes of the definition above, the geometrical structure of the array has been moved from the indexing variable to the $<ij>$ operation. Note that varying definitions of $<ij>$ and whether to "double count" generally leads to another factor $2$ difference in the numerical value of the critical temperature, $T_c$.

The coefficients are given the following names:
- $J$: Linear Interaction Coefficient
- $K$: Biquadratic Interaction Coefficient
- $D$: Anisotropy Coefficient
- $L$: Bicubic Interaction Coefficient
- $μ$: Nuclear Magnetic Moment
- $H$: External Magnetic Field Strength

Often the many of these parameters are set to $1$ or $0$, and it is common to find $K = D = L = 0$ and $J = μH = 1$ in papers that explore more exotic interactions. Additionally, it is often assumed that the coefficients are independent of the lattice sites such that $J_{ij} = J$, for example.

The _configuration probability_ $P(s, β)$ of the system then follows a Boltzmann Distribution:

$$
P(s, β) = \frac{e^{-β𝒣(s)}}{\sum_{s} e^{-β𝒣(s)}}
$$

Interestingly, we are already able to determine thermodynamic quantities about a given state $s$ by recognizing the partition function in the denominator of the configuration probability:

$$
Z(s, β) = \sum_{s} e^{-β𝒣(s)}
$$

From which we can recover the magnetisation density $M$ as (Taheridehkordi and Zivieri, 2020):

$$
M(s) = \frac{1}{L^D} \sum_{i=0}^{L^D -1} s_i
$$




## Theory - Computational

## Acknowledgements
I wish to thank Google for providing generous access to free `TPU v2-8` and `TPU v3-8` machines through their TPU Research Cloud (TRC, Google, n.d.) programme.

## References
- van der Maas, H.L.J., Dalege, J., Waldorp, L., 2020. The polarization within and across individuals: the hierarchical Ising opinion model. Journal of Complex Networks 8, cnaa010. https://doi.org/10.1093/comnet/cnaa010
- XLA, Google, n.d. XLA: Optimizing Compiler for Machine Learning [WWW Document]. TensorFlow. URL https://www.tensorflow.org/xla (accessed 4.21.23).
- TPU, Google, n.d. Train and run machine learning models faster | Cloud TPU [WWW Document]. Google Cloud. URL https://cloud.google.com/tpu (accessed 4.21.23).
- TRC, Google, n.d. TPU Research Cloud - About [WWW Document]. URL https://sites.research.google/trc/about/ (accessed 4.21.23).
- Taheridehkordi, A., Zivieri, R., 2020. Critical behavior of the classical spin-1 Ising model: a combined low-temperature series expansion and Metropolis Monte Carlo analysis. arXiv:2007.08593 [cond-mat].



